In [1]:
from __future__ import unicode_literals
import requests
import json
import csv
import matplotlib.pyplot as plt
from sets import Set
import numpy as np
import cPickle

/home/udocker/phoenix-worker/environments/python2/lib/python2.7/site-packages/ipykernel/__main__.py:6: DeprecationWarning: the sets module is deprecated


https://www.lucidchart.com/documents/edit/b0ad9829-64fb-4411-940d-8faa76d63a48/0

In [2]:
def phid_to_taskid(phid):
    data = [
          ('api.token', 'cli-XXX'),
          ('phids[0]', phid),
        ]
    response = requests.post('https://code.uberinternal.com/api/phid.query', data=data)
    jresponse = json.loads(response.text)
    return jresponse['result'][phid]['name']

In [3]:
def phid_to_duplicate(phid):
    data = [
          ('sourcePHIDs[0]', phid),
          ('types[0]', "task.merged-in"),
          ('types[1]', "task.duplicate"),
          ('api.token', 'cli-XXX')
        ]
    response = requests.post('https://code.uberinternal.com/api/edge.search', data=data)
    jresponse = json.loads(response.text)
    return jresponse['result']['data']

In [4]:
def taskid_to_phid(taskid):
    data = [
          ('api.token', 'cli-XXX'),
          ('task_id', taskid),
        ]
    response = requests.post('https://code.uberinternal.com/api/maniphest.info', data=data)
    jresponse = json.loads(response.text)
    return jresponse['result']['phid']

In [5]:
def taskid_to_content(taskid):
    data = [
          ('api.token', 'cli-XXX'),
          ('task_id', taskid),
        ]
    response = requests.post('https://code.uberinternal.com/api/maniphest.info', data=data)
    jresponse = json.loads(response.text)
    return jresponse

In [6]:
def convert_to_taskid_in_duplicate(duplicate):
    dup = []
    for d in duplicate: 
        destinationTaskID = phid_to_taskid(d['destinationPHID'])
        dup.append(str(destinationTaskID[1:]))
    return dup

In [7]:
def taskid_to_issnap(taskid):
    datat = [
              ('api.token', 'cli-XXX'),
              ('task_id', taskid),
            ]
    response = requests.post('https://code.uberinternal.com/api/maniphest.info', data=datat)
    jresponse = json.loads(response.text)
    return 'snaptask' in jresponse['result']['description']

---

## test util functions

In [8]:
content = taskid_to_content('1307739')

In [8]:
def parse_description(content):
    desc = content['result']['description']
    d = desc.split('\n')
    app_info = {}
    for i in d:
        if 'App Name:' in i:
            app_info['App Name'] =  i.split(':')[1].strip()
        if 'Version:' in i:
            app_info['Version'] =  i.split(':')[1].strip()
        if 'OS Version:' in i:
            app_info['OS Version'] =  i.split(':')[1].strip()
        if 'City:' in i:
            app_info['City'] =  i.split(':')[1].strip()
        if 'Locale:' in i:
            app_info['Locale'] =  i.split(':')[1].strip()
        if 'Device:' in i:
            app_info['Device'] =  i.split(':')[1].strip()
        if 'Vehicle:' in i:
            app_info['Vehicle'] =  i.split(':')[1].strip()
        if 'User Id:' in i:
            app_info['User Id'] =  i.split(':')[1].strip()
    return app_info

In [ ]:
duplicate = phid_to_duplicate(phid)
duplicate

In [ ]:
dup = convert_to_taskid_in_duplicate(duplicate)
dup

In [ ]:
phid_to_taskid(phid)

In [ ]:
a = Set([Set([u'1496005', u'1496217']),
 Set([u'1667117', u'1253979'])])

In [ ]:
len(a)

In [ ]:
b = Set([Set([u'1496005', u'1496217']),
 Set([u'1496217', u'1496005'])])

In [ ]:
len(b)

---

## find the merged tasks

In [10]:
dup_dict ={}
ct = 0
with open("taskid_list_6-7-2018.txt") as f:
    for line in f:
        ct = ct + 1
        taskid = line[1:-2]
        #print taskid
        phid = taskid_to_phid(taskid)
        #print phid
        duplicate = phid_to_duplicate(phid)
        duplicate = convert_to_taskid_in_duplicate(duplicate)
        dup_dict[taskid] = duplicate
        print dup_dict
        if ct>5:
            break

{'1807737': []}
{'1807657': [], '1807737': []}
{'1807623': [], '1807657': [], '1807737': []}
{'1807623': [], '1807657': [], '1807575': [], '1807737': []}
{'1807623': [], '1807657': [], '1807797': [], '1807575': [], '1807737': []}
{'1807623': [], '1807657': [], '1807653': [], '1807575': [], '1807797': [], '1807737': []}


In [24]:
phid = taskid_to_phid('1858711')

In [25]:
phid

u'PHID-TASK-6orhcz7hvpctibl2o4ge'

In [26]:
duplicate = phid_to_duplicate(phid)
duplicate = convert_to_taskid_in_duplicate(duplicate)

In [27]:
duplicate

[]

In [9]:
jresponse = json.loads(response.text)

In [ ]:
with open('pair_list_6-7-2018.json', 'w') as fp:
    json.dump(dup_dict, fp)

---

## analyze duplicates

In [9]:
with open('pair_list_1-5-2018.json') as json_file:  
    data = json.load(json_file)

In [10]:
print 'total number of reports from snaptask within 5 months:', len(data)

total number of reports from snaptask within 5 months: 17434


In [11]:
# create set_list from duplicate key value pair
## remove the pair that contains ticket outside snaptask 1-5 2018
set_list = []
for k in data.keys():
    for i in data[k]:
        if i in data.keys():
            set_list.append(Set([k,i]))

In [12]:
print 'total nubmer of pairs: ', len(set_list)

total nubmer of pairs:  4444


In [13]:
print 'total nubmer of unique pairs: ', len(Set(set_list))

total nubmer of unique pairs:  2236


In [ ]:
# find tickets not from snaptask 1-5 2018
## manually created master tasks
## snaptask tickets created outside this time range 
m_list = []
for k in data.keys():
    for i in data[k]:
        if i not in data.keys():
            m_list.append(i)

In [30]:
print 'tickets outside snaptask: ', len(set(m_list))

tickets outside snaptask:  825


use phab info to find snap ticket

In [ ]:
manual_list = []
snap_list = []
for i in set(m_list):
    js = taskid_to_issnap(i)
    if js is True:
        snap_list.append(i)
    else:
        manual_list.append(i)

In [ ]:
len((snap_list))

In [ ]:
len((manual_list))

In [ ]:
manual_list

In [245]:
316+509

825

### adhoc solution to find snap ticket

In [15]:
manual_list = [ i for i in m_list if i[0]<>'1']

In [16]:
snap_list = [ i for i in m_list if i[0]=='1']

In [18]:
print 'tickets outside snaptask, manually created masters: ', len(set(manual_list))

tickets outside snaptask, manually created masters:  46


In [19]:
print 'tickets outside snaptask, outside date range: ', len(set(snap_list))

tickets outside snaptask, outside date range:  737


In [39]:
# for each m_master, get the snap tickets that linked with them
ct_manual = []
for m in set(manual_list):
    merged = [key for key, value in data.iteritems() if m in value]
    if len(merged)>120:
        print m
    ct_manual.append(len(merged)-1)

698886
731515


In [21]:
# for each m_master, get the snap tickets that linked with them
for m in set(manual_list):
    merged = [key for key, value in data.iteritems() if m in value]
    merged_key = merged[0]
    for i in range(len(merged)-1):
        set_list.append(Set([merged_key,merged[i+1]]))

In [22]:
print 'total number of unique pairs after adding the manual merge:', len(Set(set_list))

total number of unique pairs after adding the manual merge: 2847


In [251]:
3894-2236

1658

In [39]:
sum(ct_manual)

611

In [23]:
# for each snap ticket outside date range, get the snap tickets that linked with them
ct_snap = []
for m in set(snap_list):
    merged = [key for key, value in data.iteritems() if m in value]
    ct_snap.append(len(merged))

In [24]:
# for each snap ticket outside date range, get the snap tickets that linked with them
for m in set(snap_list):
    merged = [key for key, value in data.iteritems() if m in value]
    for i in merged:
        set_list.append(Set([m , i]))

In [25]:
print 'total number of unique pairs after adding the manual merge:', len(Set(set_list))

total number of unique pairs after adding the manual merge: 5079


In [26]:
len(list(Set(set_list)))

5079

In [27]:
L = []
for l in list(set_list):
    L = L + list(l)

In [28]:
len(set(L))

6184

In [29]:
pair_training = []
for l in Set(set_list):
    lst = list(l)
    pair_training.append(lst)

In [47]:
len(pair_training)

5122

In [55]:
cPickle.dump(pair_training, open('6-7_2018_duplicated_pairs.p', 'wb')) 

In [316]:
obj = cPickle.load(open('1-5_2018_duplicated_pairs.p', 'rb'))

In [38]:
ct = []
for k in data.keys():
    tmp = 0
    for i in data[k]:
        if i in data.keys():
            tmp = tmp+1
            if tmp> 99:
                print k, i
    ct.append(tmp)

1646167 1646159


In [35]:
max(ct_snap)

93

In [34]:
max(ct_manual)

126

In [36]:
max(ct)

100

In [49]:
ct = ct + ct_snap + ct_manual

In [50]:
print 'max duplicate: ', max(ct)
ct = [i for i in ct if i > 0]
print 'min duplicate: ', min(ct)
print 'total number of reports that have duplicates: ', len(ct)
print 'total number of reports: ', (len(data)+len(snap_list))
print '%: ', float(len(ct)*2)/(len(data)+len(snap_list))
print 'average duplicates: ', np.average(ct)
print 'median duplicates: ', np.median(ct)
print 'p95 duplicates: ', np.percentile(ct, 95)
print 'total number of duplicated pairs: ', np.sum(ct)

print 'total number of unique duplicated pairs: ', len(Set(set_list))
print '%: ', float(len(Set(set_list))*2)/(len(data)+len(snap_list))

max duplicate:  126
min duplicate:  1
total number of reports that have duplicates:  3786
total number of reports:  19721
%:  0.383956188834
average duplicates:  1.93608029583
median duplicates:  1.0
p95 duplicates:  4.0
total number of duplicated pairs:  7330
total number of unique duplicated pairs:  5122
%:  0.519446275544


In [276]:
(len(data)+len(snap_list))

17750

In [ ]:
plt.figure(figsize=[20,3])
n, bins, patches = plt.hist(ct, 100, facecolor='blue', alpha=0.5)
plt.show()

In [ ]:
ct = [i for i in ct if i > 0]
plt.figure(figsize=[20,3])
n, bins, patches = plt.hist(ct, 100, facecolor='blue', alpha=0.5)
plt.show()

In [ ]:
ct = [i for i in ct if i <20]
plt.figure(figsize=[20,3])
n, bins, patches = plt.hist(ct, 100, facecolor='blue', alpha=0.5)
plt.show()

---